In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
sample_submission = pd.read_csv('data/sample_submission.csv')

In [27]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

from sklearn.metrics import mean_squared_error, r2_score

merged_df = pd.read_csv('data/train_data2.csv').drop('Unnamed: 0', axis=1)

X, y = merged_df.drop(['stock_id','time_id','target'], axis=1), merged_df['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [5]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 428932 entries, 0 to 428931
Data columns (total 18 columns):
stock_id          428932 non-null float64
time_id           428932 non-null float64
total_ratio       428932 non-null float64
real_vol          428932 non-null float64
skewness          428932 non-null float64
right_skew        428932 non-null float64
book_row_num      428932 non-null float64
bid_size1         428932 non-null float64
ask_size1         428932 non-null float64
bid_size2         428932 non-null float64
ask_size2         428932 non-null float64
traded_size       428932 non-null float64
trade_TR          428932 non-null float64
max_size          428932 non-null float64
max_size_price    428932 non-null float64
max_price         428932 non-null float64
max_price_size    428932 non-null float64
target            428932 non-null float64
dtypes: float64(18)
memory usage: 58.9 MB


#### 0. Learning_rate : initial value 0.3

#### 1. n_estimators tuning

In [14]:
xgbr = XGBRegressor(objective = 'reg:squarederror')
n_estimators = list(range(50, 400, 50))
param_grid = {'n_estimators' : n_estimators}

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 8)
grid_search.fit(X_train, y_train)
grid_search.cv_results_

Fitting 8 folds for each of 7 candidates, totalling 56 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

{'mean_fit_time': array([12.46824837, 26.83711147, 37.59500429, 47.0774034 , 61.49033272,
        82.39780587, 87.33136669]),
 'std_fit_time': array([0.65215304, 1.74199071, 5.693142  , 1.26744934, 2.71142286,
        6.34874659, 4.49026539]),
 'mean_score_time': array([0.03903213, 0.09786671, 0.15091649, 0.1811296 , 0.23961878,
        0.32467231, 0.33663398]),
 'std_score_time': array([0.0033344 , 0.00881749, 0.02258506, 0.01055909, 0.03500746,
        0.01822163, 0.02720402]),
 'param_n_estimators': masked_array(data=[50, 100, 150, 200, 250, 300, 350],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_estimators': 50},
  {'n_estimators': 100},
  {'n_estimators': 150},
  {'n_estimators': 200},
  {'n_estimators': 250},
  {'n_estimators': 300},
  {'n_estimators': 350}],
 'split0_test_score': array([-8.30273301e-06, -1.75925827e-06, -1.75235352e-06, -1.74760139e-06,
        -1.74543959e-06, -1.743892

In [19]:
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

,params,mean_test_score,rank_test_score
0,{'n_estimators': 50},-0.000008,7
1,{'n_estimators': 100},-0.000002,6
2,{'n_estimators': 150},-0.000002,5
3,{'n_estimators': 200},-0.000002,4
4,{'n_estimators': 250},-0.000002,3
5,{'n_estimators': 300},-0.000002,2
6,{'n_estimators': 350},-0.000002,1


In [20]:
# n_estimators : 100으로 일단. 

#### 2. max_depth tuning

In [23]:
xgbr = XGBRegressor(objective = 'reg:squarederror', n_estimators = 100)
max_depth = np.arange(1, 11, 2)
param_grid = dict(max_depth = max_depth)

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 8)
grid_search.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

Fitting 8 folds for each of 5 candidates, totalling 40 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

,params,mean_test_score,rank_test_score
0,{'max_depth': 1},-0.000002,5
1,{'max_depth': 3},-0.000002,4
2,{'max_depth': 5},-0.000002,2
3,{'max_depth': 7},-0.000002,1
4,{'max_depth': 9},-0.000002,3


In [1]:
# max_depth는 7이 제일 좋았고, 5, 7, 9로 후보를 좁혀서 min_child_weight랑 함께

#### 3. min_child_weight tuning : among 1, 3, 5

In [29]:
xgbr = XGBRegressor(objective = 'reg:squarederror', n_estimators = 100)
max_depth = np.array([5, 7, 9])
min_child_weight = np.arange(1, 6, 2)
param_grid = dict(max_depth = max_depth, min_child_weight = min_child_weight)

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 8)
grid_search.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

Fitting 8 folds for each of 9 candidates, totalling 72 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

,params,mean_test_score,rank_test_score
0,"{'max_depth': 5, 'min_child_weight': 1}",-0.000002,7
1,"{'max_depth': 5, 'min_child_weight': 3}",-0.000002,6
2,"{'max_depth': 5, 'min_child_weight': 5}",-0.000002,8
3,"{'max_depth': 7, 'min_child_weight': 1}",-0.000002,5
4,"{'max_depth': 7, 'min_child_weight': 3}",-0.000002,3
5,"{'max_depth': 7, 'min_child_weight': 5}",-0.000002,1
6,"{'max_depth': 9, 'min_child_weight': 1}",-0.000002,9
7,"{'max_depth': 9, 'min_child_weight': 3}",-0.000002,4
8,"{'max_depth': 9, 'min_child_weight': 5}",-0.000002,2


In [2]:
# max_depth = 7, min_child_weight = 5 사용

#### 4. subsample, colsample_bytree tuning

In [8]:
xgbr = XGBRegressor(objective = 'reg:squarederror', 
                    n_estimators = 100,
                    max_depth = 7,
                    min_child_weight = 5)
param_grid = {
 'subsample':[i/10.0 for i in range(6,10)],
 'colsample_bytree':[i/10.0 for i in range(6,10)]
}

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 6)
grid_search.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

Fitting 6 folds for each of 16 candidates, totalling 96 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

,params,mean_test_score,rank_test_score
0,"{'colsample_bytree': 0.6, 'subsample': 0.6}",-0.000002,13
1,"{'colsample_bytree': 0.6, 'subsample': 0.7}",-0.000002,7
2,"{'colsample_bytree': 0.6, 'subsample': 0.8}",-0.000002,3
3,"{'colsample_bytree': 0.6, 'subsample': 0.9}",-0.000002,4
4,"{'colsample_bytree': 0.7, 'subsample': 0.6}",-0.000002,12
5,"{'colsample_bytree': 0.7, 'subsample': 0.7}",-0.000002,2
6,"{'colsample_bytree': 0.7, 'subsample': 0.8}",-0.000002,6
7,"{'colsample_bytree': 0.7, 'subsample': 0.9}",-0.000002,1
8,"{'colsample_bytree': 0.8, 'subsample': 0.6}",-0.000002,15
9,"{'colsample_bytree': 0.8, 'subsample': 0.7}",-0.000002,9


In [9]:
# colsample_bytree = 0.7, subsample = 0.9

#### 5. gamma tuning ( + alpha)

In [4]:
xgbr = XGBRegressor(objective = 'reg:squarederror', 
                    n_estimators = 100,
                    max_depth = 7,
                    min_child_weight = 5,
                    colsample_bytree = 0.7,
                    subsample = 0.9)

param_grid = {
    'gamma':[i/10.0 for i in range(0,5)],
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 6)
grid_search.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

Fitting 6 folds for each of 25 candidates, totalling 150 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

,params,mean_test_score,rank_test_score
0,"{'gamma': 0.0, 'reg_alpha': 1e-05}",-0.000002,3
1,"{'gamma': 0.0, 'reg_alpha': 0.01}",-0.000002,1
2,"{'gamma': 0.0, 'reg_alpha': 0.1}",-0.000002,2
3,"{'gamma': 0.0, 'reg_alpha': 1}",-0.000002,4
4,"{'gamma': 0.0, 'reg_alpha': 100}",-0.000002,5
5,"{'gamma': 0.1, 'reg_alpha': 1e-05}",-0.000003,9
6,"{'gamma': 0.1, 'reg_alpha': 0.01}",-0.000003,7
7,"{'gamma': 0.1, 'reg_alpha': 0.1}",-0.000003,6
8,"{'gamma': 0.1, 'reg_alpha': 1}",-0.000003,8
9,"{'gamma': 0.1, 'reg_alpha': 100}",-0.000003,10


In [5]:
# 'gamma': 0.0, 'reg_alpha': 0.01

#### 6. learning_rate tuning

In [7]:
xgbr = XGBRegressor(objective = 'reg:squarederror', 
                    n_estimators = 100,
                    max_depth = 7,
                    min_child_weight = 5,
                    colsample_bytree = 0.7,
                    subsample = 0.9,
                    gamma = 0,
                    reg_alpha = 0.01)

learning_rate = np.arange(0.3, 0, -0.05)
param_grid = dict(learning_rate = learning_rate)

grid_search = GridSearchCV(xgbr, param_grid, scoring = 'neg_mean_squared_error', verbose=1, cv = 6)
grid_search.fit(X_train, y_train)
scores_df = pd.DataFrame(grid_search.cv_results_)
scores_df[['params','mean_test_score','rank_test_score']]

Fitting 6 folds for each of 6 candidates, totalling 36 fits


C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaconda3\lib\site-packages\xgboost\core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
C:\Users\frank\Anaco

,params,mean_test_score,rank_test_score
0,{'learning_rate': 0.3},-0.000002,5
1,{'learning_rate': 0.25},-0.000002,4
2,{'learning_rate': 0.2},-0.000002,3
3,{'learning_rate': 0.15000000000000002},-0.000002,2
4,{'learning_rate': 0.10000000000000003},-0.000002,1
5,{'learning_rate': 0.050000000000000044},-0.000010,6


In [8]:
# learning_rate = 0.1일 때 가장 좋았음

#### Final Model

In [28]:
best_xgbr = XGBRegressor(objective = 'reg:squarederror',
                    learning_rate = 0.1,
                    n_estimators = 100,
                    max_depth = 7,
                    min_child_weight = 5,
                    colsample_bytree = 0.7,
                    subsample = 0.9,
                    gamma = 0,
                    reg_alpha = 0.01)

best_xgbr.fit(X_train, y_train)
y_pred = best_xgbr.predict(X_test)
print('RMSE :', np.sqrt(mean_squared_error(y_test, y_pred)))
print('R2 :', r2_score(y_test, y_pred))
y_test_df = pd.DataFrame({'y_test':y_test, 'y_pred':y_pred})
y_test_df['rmspe_before'] = ((y_test_df.y_test-y_test_df.y_pred)/y_test_df.y_test)**2
print('RMSPE :',np.sqrt(y_test_df.rmspe_before.sum()/len(y_test_df)))

RMSE : 0.0013182874627396216
R2 : 0.7959968828476296
RMSPE : 0.29483755499102876


In [16]:
y_test_ser = y_test.copy()
y_test['y_pred'] = y_pred

In [18]:
y_test_df = pd.DataFrame({'y_test':y_test_ser, 'y_pred':y_pred})

In [19]:
y_test_df

,y_test,y_pred
335348,0.001080,0.001641
53138,0.002631,0.003359
53063,0.002518,0.002835
237838,0.002687,0.002098
217530,0.002332,0.002354
13632,0.004292,0.003210
167779,0.005386,0.007477
396228,0.008531,0.009414
177817,0.002661,0.002589
365732,0.001611,0.001986


In [22]:
y_test_df['rmspe_before'] = ((y_test_df.y_test-y_test_df.y_pred)/y_test_df.y_test)**2

In [25]:
np.sqrt(y_test_df.rmspe_before.sum()/len(y_test_df))

0.30317367406934387